# COVID-19 TWEET SENTIMENT CLASSIFICATION

![](https://viterbischool.usc.edu/wp-content/uploads/2020/03/COVID_TWITTER1200X600.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding='latin1')

In [ ]:
train.head()

In [ ]:
train['Location'].unique()

In [ ]:
train['Location'].value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
a=train['Sentiment'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import plotly.express as px

fig = px.bar(a, x=['Positive','Negative','Neutral','Extremely Positive','Extremely Negative'], y='Sentiment',color=['Positive','Negative','Neutral','Extremely Positive','Extremely Negative'])
fig.show()

In [ ]:
t=train['Location'].value_counts()[:10]

In [ ]:
import plotly.express as px

fig = px.bar(t,y='Location', color=t.index)
fig.show()

In [ ]:
t.values

In [ ]:
time=train['TweetAt'].to_frame

In [ ]:
y=train['Sentiment'].copy()

In [ ]:
train['Sentiment']=train['Sentiment'].map({'Positive':0,'Negative':1,'Neutral':2,'Extremely Positive':3,
                                           'Extremely Negative':4})

In [ ]:
train['OriginalTweet']

In [ ]:
train['OriginalTweet'][0]

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

import re

In [ ]:
def cleaner(tweet):
    
    # remove urls
    tweet = re.sub(r'http\S+', ' ', tweet)
    
    # remove html tags
    tweet = re.sub(r'<.*?>',' ', tweet)
    
    # remove digits
    tweet = re.sub(r'\d+',' ', tweet)
    
    # remove hashtags
    tweet = re.sub(r'#\w+',' ', tweet)
    
    # remove mentions
    tweet = re.sub(r'@\w+',' ', tweet)
    
    #removing stop words
    tweet = tweet.split()
    tweet = " ".join([word for word in tweet if not word in stop_words])
    
    return  tweet
    

In [ ]:

stop_words = stopwords.words('english')
train['OriginalTweet']=train['OriginalTweet'].apply(lambda x:x.lower())
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train_cleaned = train['OriginalTweet'].apply(cleaner)
train_cleaned.head()

In [ ]:
train['OriginalTweet']=train_cleaned

In [ ]:
import wordcloud

In [ ]:
all_words = ' '.join([text for text in train_cleaned])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

![](https://c.tenor.com/RMYmjb3OW44AAAAj/emotional-emotion.gif)

Positive

In [ ]:
Positive =' '.join([text for text in train['OriginalTweet'][train['Sentiment'] == 0]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(Positive)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Negative

In [ ]:
Negative =' '.join([text for text in train['OriginalTweet'][train['Sentiment'] == 1]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(Negative)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Neutral

In [ ]:
Neutral =' '.join([text for text in train['OriginalTweet'][train['Sentiment'] == 2]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(Neutral)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Extremely Positive

In [ ]:
Extremely_Positive =' '.join([text for text in train['OriginalTweet'][train['Sentiment'] == 3]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(Extremely_Positive)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Extremely Negative

In [ ]:
Extremely_Negative =' '.join([text for text in train['OriginalTweet'][train['Sentiment'] == 4]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(Extremely_Negative)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
train['TweetAt'].value_counts()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.layers as Layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns


import numpy as np 
import pandas as pd

In [ ]:
# Tokenizer class in tensorflow allow us to vectorize 
# a text corpus into sequence of integers

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_cleaned)

X = tokenizer.texts_to_sequences(train_cleaned)

vocab_size = len(tokenizer.word_index)+1



print("Vocabulary size: {}".format(vocab_size))
print("\nExample:\n")
print("Sentence:\n{}".format(train_cleaned[6]))
print("\nAfter tokenizing :\n{}".format(X[6]))

X = pad_sequences(X, padding='post')
print("\nAfter padding :\n{}".format(X[6]))

In [ ]:
encoding = {'Extremely Negative': 0,
            'Negative': 0,
            'Neutral': 1,
            'Positive':2,
            'Extremely Positive': 2
           }

labels = ['Negative', 'Neutral', 'Positive']
           
y.replace(encoding, inplace=True)

In [ ]:
tf.keras.backend.clear_session()

# hyper parameters
EPOCHS = 2
BATCH_SIZE = 32
embedding_dim = 16
units = 256

model = tf.keras.Sequential([
    Layers.Embedding(vocab_size, embedding_dim, input_length=X.shape[1]),
    Layers.Bidirectional(Layers.LSTM(units,return_sequences=True)),
    Layers.GlobalMaxPool1D(),
    Layers.Dropout(0.4),
    Layers.Dense(64, activation="relu"),
    Layers.Dropout(0.4),
    Layers.Dense(3)
])


model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',metrics=['accuracy']
             )

model.summary()

Note:-Here I have used Bidirectional LSTM.
Bidirectional LSTMs are an extension to typical LSTMs that can enhance performance of the model on sequence classification problems. Where all time steps of the input sequence are available, Bi-LSTMs train two LSTMs instead of one LSTMs on the input sequence.
![](https://www.i2tutorials.com/wp-content/media/2019/05/Deep-Dive-into-Bidirectional-LSTM-i2tutorials.jpg)

In [ ]:
history = model.fit(X, y, epochs=EPOCHS, validation_split=0.12, batch_size=BATCH_SIZE)

In [ ]:
fig = px.line(
    history.history, y=['accuracy', 'val_accuracy'],
    labels={'index': 'epoch', 'value': 'accuracy'}
)

fig.show()


In [ ]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}
)

fig.show()

In [ ]:
test=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_test.csv")

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
X_test = test['OriginalTweet'].copy()
y_test = test['Sentiment'].copy()

X_test = X_test.apply(cleaner)

X_test = tokenizer.texts_to_sequences(X_test)

X_test = pad_sequences(X_test, padding='post')

y_test.replace(encoding, inplace=True)

In [ ]:
pred = model.predict_classes(X_test)

In [ ]:
pred

In [ ]:
loss, acc = model.evaluate(X_test,y_test,verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

In [ ]:
conf = confusion_matrix(y_test, pred)

cm = pd.DataFrame(
    conf, index = [i for i in labels],
    columns = [i for i in labels]
)

plt.figure(figsize = (12,7))
sns.heatmap(cm, annot=True, fmt="d")
plt.show()

In [ ]:
print(classification_report(y_test, pred, target_names=labels))

<span class='h1'>Please Upvote if you liked the work</span>

In [ ]:
%%writefile notebook_style.py
from IPython.core.display import display,HTML

def notebook_style(span_h_font='Ewert',
                   span_h_color='#ff3636',
                   prompt_font='Ewert',
                   prompt_in_color='#ff3636',
                   prompt_out_color='#3636ff',
                   warn_font='Roboto',
                   warn_color='#ff36ff',
                   out_font='Roboto',
                   out_color='#3636ff',
                   background_out_color='whitesmoke',
                   span_text_shadow=True):
    style_str="""<style>"""+\
    """@import 'https://fonts.googleapis.com/css?family="""+\
    prompt_font+"""|"""+out_font+"""|"""+warn_font+"""'; """
    if span_text_shadow==True:
        style_str+="""span {color:black; """+\
                   """text-shadow:4px 4px 4px #aaa;}"""
    style_str+="""div.alert {text-shadow:4px 4px 4px #aaa;}"""
    style_str+="""span.h1,span.h2,span.h3,"""+\
               """span.h4,span.h5,span.h6 """+\
               """{color:"""+span_h_color+"""; """+\
               """font-family:"""+span_h_font+""";} """
    style_str+="""div.warn {background-color:"""+\
                            background_out_color+"""; color:"""+\
               warn_color+"""; font-size:110%; """+\
               """font-family:"""+warn_font+""";} """
    for el in ["""div.output_area pre""",
               """div.output_stderr pre"""
               """div.output_subarea""",
               """div.output_html""",
               """div.output_stderr"""]:
        style_str+=el+"""{background-color:"""+\
                   background_out_color+"""; color:"""+\
                   out_color+"""; font-size:110%; """+\
                   """font-family:"""+out_font+""";} """
    style_str+="""div.input_prompt {color:"""+\
               prompt_in_color+"""; """+\
               """font-family:"""+prompt_font+""";} """
    style_str+="""div.output_prompt {color:"""+\
               prompt_out_color+"""; """+\
               """font-family:"""+prompt_font+""";} """
    style_str+=""".cm-s-ipython span.cm-comment {color:darkslategray;} """
    style_str+=""".cm-s-ipython span.cm-def {color:#3636ff;} """
    style_str+=""".cm-s-ipython span.cm-operator {color:#ff36ff;} """
    style_str+=""".cm-s-ipython span.cm-keyword {color:darkgreen;} """
    style_str+=""".cm-s-ipython span.cm-string {color:#ff3636;} """
    style_str+="""</style>"""
    display(HTML(style_str))

In [ ]:
%run notebook_style.py

# default options
notebook_style(span_h_font='Ewert',
               span_h_color='#ff3636',
               prompt_font='Ewert',
               prompt_in_color='#ff3636',
               prompt_out_color='#3636ff',
               warn_font='Roboto',
               warn_color='#ff36ff',
               out_font='Roboto',
               out_color='#3636ff',
               background_out_color='whitesmoke',
               span_text_shadow=True)

I have learned this kind of styling from this notebook:
https://www.kaggle.com/olgabelitskaya/styling-recipes
This notebook by **Olga Belitskaya** is truely awesome!!